In [1]:
import numpy as np
import tensorflow as tf

from typing import List, Container
from tensorflow.data import Dataset
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Concatenate, Dense, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import Mean

class TextCNN(Model):
    def __init__(self, class_num: int):
        super(TextCNN, self).__init__()
        self.class_num: int = class_num
        
    def build(self, input_shape):
        self.conv_1: Conv1D = Conv1D(filters=128, kernel_size=1, activation="relu", name="conv_1")
        self.pool_1: MaxPool1D = MaxPool1D(pool_size=2, strides=1,name="pool_1")
        self.conv_2: Conv1D = Conv1D(filters=128, kernel_size=2, activation="relu", name="conv_2")
        self.pool_2: MaxPool1D = MaxPool1D(pool_size=2, strides=1, name="pool_2")
        self.concatenate: Concatenate = Concatenate(axis=1)
        self.flatten: Flatten = Flatten()
        self.dense: Dense = Dense(self.class_num, activation="softmax")
        super(TextCNN, self).build(input_shape)

    def call(self, inputs: Dataset, training=None, mask=None):
        convs: List[Conv1D] = [self.conv_1(inputs), self.conv_2(inputs)]
        pools: List[MaxPool1D] = [self.pool_1(convs[0]), self.pool_2(convs[1])]
        x = self.concatenate(pools)
        x = self.flatten(x)
        x = self.dense(x)
        return x
    
    def summary(self):
        input: Input = Input(shape=(3, 100), name="Input")
        output = self.call(input)
        model = Model(inputs=input, outputs=output, name="TextCNN")
        model.summary()


In [9]:
from user_profile_prediction.etl.preprocess_train_data import PreprocessTrainingData
from user_profile_prediction.etl.embedding import Embedding
p = PreprocessTrainingData("/Volumes/Samsung_T5/Files/Document/小象学院/GroupProject/project_data/data/train.csv")
p.split_sentence()

e = Embedding(100, 5)
m = e.load_embedding_model()

/Users/luominzhi/Code/Python/user_profile_prediction/user_profile_prediction/etl/preprocess_train_data.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df: DataFrame = pd.read_csv(file_path, sep="###__###", header=None)
0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/zl/nw7xtsq52579dn6nkhtf8njh0000gn/T/jieba.cache
Loading model cost 0.727 seconds.
Prefix dict has been built successfully.
1000it [00:55, 18.15it/s]


In [10]:
train_x, train_y = [], []

for x, y in p.age_data_iter(e):
    train_x.append(x)
    train_y.append(y)

train_x, train_y = np.array(train_x).astype(np.float32), np.array(train_y)
train_y = tf.one_hot(train_y, depth=tf.unique(train_y).y.shape[0])

In [11]:
text_cnn = TextCNN(7)

optimizer: Adam = Adam(learning_rate=1e-3)
losses: CategoricalCrossentropy = CategoricalCrossentropy()

train_loss = Mean(name="TrainLoss")

In [12]:
text_cnn.build(input_shape=(None, 3, 100))
text_cnn.summary()



Model: "TextCNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 3, 100)]     0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 3, 128)       12928       Input[0][0]                      
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 2, 128)       25728       Input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling1D)           (None, 2, 128)       0           conv_1[0][0]                     
____________________________________________________________________________________________

In [13]:
text_cnn(train_x[:2])

<tf.Tensor: shape=(2, 7), dtype=float32, numpy=
array([[0.14571063, 0.14675926, 0.14144206, 0.1375995 , 0.14187463,
        0.14424293, 0.14237098],
       [0.13359731, 0.17813636, 0.1637587 , 0.11939705, 0.18103491,
        0.11319735, 0.11087842]], dtype=float32)>

In [14]:
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        prediction = model(features)
        loss = losses(labels, prediction)
    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    
    train_loss.update_state(loss)
    
    return gradient

In [15]:
g = train_step(text_cnn, train_x[:10], train_y[:10])


In [16]:
g


[<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
 array([[[-2.1848297e-02, -1.1947261e-02, -2.6903633e-04, ...,
           0.0000000e+00, -2.5617115e-02,  0.0000000e+00],
         [ 1.5136064e-03,  3.9455388e-03,  1.1869123e-04, ...,
           0.0000000e+00, -8.4274798e-04,  0.0000000e+00],
         [ 4.9521928e-03,  1.1383957e-03, -1.3643265e-05, ...,
           0.0000000e+00,  4.3556560e-03,  0.0000000e+00],
         ...,
         [ 3.2905366e-02,  1.5232649e-02,  3.2045774e-04, ...,
           0.0000000e+00,  3.5915498e-02,  0.0000000e+00],
         [-3.9836656e-02, -2.0150166e-02, -4.2761990e-04, ...,
           0.0000000e+00, -4.2372078e-02,  0.0000000e+00],
         [-1.2364896e-02, -6.5314802e-03, -2.0052551e-04, ...,
           0.0000000e+00, -1.3592942e-02,  0.0000000e+00]]], dtype=float32)>,
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.08890758,  0.05574488,  0.00982114,  0.        ,  0.        ,
         0.00659037,  0.10970964,  0.        ,  0.       

In [17]:
text_cnn.compile(optimizer=optimizer, loss=losses)
history = text_cnn.fit(train_x, train_y, epochs=5, batch_size=100)


Epoch 1/5
1478/1478 [==============================] - 2s 1ms/step - loss: 1.4563

In [15]:
tf.one_hot(train_y, depth=tf.unique(train_y).y.shape[0])

<tf.Tensor: shape=(147726, 7), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)>

In [19]:
from sklearn.model_selection import train_test_split

print(train_x.shape)
print(train_y.numpy().shape)

ds = tf.data.Dataset.from_tensor_slices((train_x, train_y.numpy()))

(147726, 3, 100)
(147726, 7)


In [36]:
tf.split(
    train_y.numpy(), 
    num_or_size_splits=[4, 147722],
    # num=4, 
    axis=0
)


[<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(147722, 7), dtype=float32, numpy=
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)>]